### Dataset Setup

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# Basic imports
import hvplot.xarray
import numpy as np
import xarray as xr
import xclim.indices.stats as stats
import xdatasets as xd
from scipy.stats import genextreme, genpareto, gumbel_r

import xhydro as xh
import xhydro.frequency_analysis as xhfa
from xhydro.extreme_value_analysis.parameterestimation import *

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
ds = (
    xd.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["020*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {
        "season": ["DJF"],
        "freq": "YS-DEC",
    },  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {},
}
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

# Create a mask beforehand
import random

nyears = np.unique(ds.time.dt.year).size
dom_start = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)
dom_end = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)

mask = xr.zeros_like(ds["streamflow"])
for y in dom_start.year.values:
    # Random mask of dates per year, between April and June.
    mask.loc[
        {
            "time": slice(
                str(y) + "-04-" + str(dom_start.sel(year=y).item()),
                str(y) + "-06-" + str(dom_end.sel(year=y).item()),
            )
        }
    ] = 1
# The name of the indexer will be used to identify the variable created here
timeargs_custom = {"custom": {}}

# We use where() to mask the data that we want to ignore
masked = ds.where(mask == 1)
# Since we masked almost all of the year, our tolerance for missing data should be changed accordingly
missing = "at_least_n"
missing_options = {"n": 45}

# We use xr.merge() to combine the results with the previous dataset.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            masked,
            op="max",
            timeargs=timeargs_custom,
            missing=missing,
            missing_options=missing_options,
        ),
    ]
)
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            ds,
            op="sum",
            input_var="volume",
            timeargs=timeargs_vol,
            missing="pct",
            missing_options={"tolerance": 0.15},
            interpolate_na=True,
        ),
    ]
)

# for id in range(len(ds_4fa.coords["id"])):
#     print(ds_4fa.streamflow_max_spring[id,:].values)

print(ds_4fa.streamflow_max_summer[0, :].values)

ds_4fa

[   nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
  31.4   38.8   63.78  54.82 123.2  108.2  108.1   98.52  42.24  33.45
  60.    84.23  48.07  77.71  38.37  42.02  46.      nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan]


<xarray.Dataset> Size: 14kB
Dimensions:                (id: 5, time: 56)
Coordinates: (12/15)
    drainage_area          (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date               (id) datetime64[ns] 40B 2006-10-13 ... 1996-08-13
  * id                     (id) object 40B '020302' '020404' ... '020802'
    latitude               (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude              (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name                   (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 40B 1989-08-12 ... 1970-01-01
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * time                   (time) datetime64[ns] 448B 1970-01-01 ... 2025-01-01
Data variables:
    streamflow_max_spring  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_summer  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_fall    (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_august  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_annual  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_winter  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_custom  (id, time) float32 1kB nan nan nan ... nan nan nan
    volume_sum_spring      (id, time) float64 2kB nan nan nan ... nan nan nan
    volume_sum_annual      (id, time) float64 2kB nan nan nan ... nan nan nan
Attributes:
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [3]:
import glob
import os

from xarray import open_zarr

# path_hist = glob.glob(os.path.join(r"C:\Users\KAMIL PC\Desktop\Life\Stage\Documents\non_stationary_data\eau_precpitable_CanESM5"))
path_hist = glob.glob(
    os.path.join(
        r"C:\Users\HP\Desktop\Stage\non_stationary_data\eau_precpitable_CanESM5"
    )
)

ds_hist = open_zarr(path_hist[0], consolidated=True)
ds_hist

<xarray.Dataset> Size: 326kB
Dimensions:             (time: 2712, y: 4, x: 7)
Coordinates:
    height              float64 8B ...
  * time                (time) object 22kB 1925-01-31 00:00:00 ... 2150-12-31...
  * x                   (x) float64 56B 275.6 278.4 281.2 ... 286.9 289.7 292.5
  * y                   (y) float64 32B 46.04 48.84 51.63 54.42
Data variables:
    precipitable_water  (time, y, x) float32 304kB dask.array<chunksize=(2712, 4, 7), meta=np.ndarray>

# extreme_value_analysis.fit()

In [4]:
params = fit(ds_4fa, dist=genextreme, method="ml")
# print(type(params.precipitable_water[:, 0, 0]))
params

<xarray.Dataset> Size: 4kB
Dimensions:                (id: 5, dparams: 9)
Coordinates: (12/15)
    drainage_area          (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date               (id) datetime64[ns] 40B 2006-10-13 ... 1996-08-13
  * id                     (id) object 40B '020302' '020404' ... '020802'
    latitude               (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude              (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name                   (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 40B 1989-08-12 ... 1970-01-01
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * dparams                (dparams) <U11 396B 'shape' ... 'scale_upper'
Data variables:
    streamflow_max_spring  (id, dparams) float64 360B nan nan ... 51.45 98.12
    streamflow_max_summer  (id, dparams) float64 360B 0.4454 -0.2299 ... 88.42
    streamflow_max_fall    (id, dparams) float64 360B 0.8062 -0.2569 ... 27.84
    streamflow_max_august  (id, dparams) float64 360B 0.9015 0.01491 ... 18.39
    streamflow_max_annual  (id, dparams) float64 360B nan nan ... 51.38 100.1
    streamflow_max_winter  (id, dparams) float64 360B nan nan ... 6.664 17.17
    streamflow_max_custom  (id, dparams) float64 360B -0.3362 -0.9512 ... 103.2
    volume_sum_spring      (id, dparams) float64 360B -0.6526 -1.086 ... 174.3
    volume_sum_annual      (id, dparams) float64 360B -0.2645 -0.5601 ... 218.5
Attributes:
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:                
    long_name:             genextreme parameters
    description:           Parameters of the genextreme distribution
    method:                ML
    estimator:             Maximum likelihood
    scipy_dist:            genextreme
    units:                 
    history:               [2024-07-22 09:52:36] fit: Estimate distribution p...

# xclim.stats.fit()

In [5]:
from lmoments3.distr import gev, gpa, gum

params = stats.fit(ds_4fa, dist=gev, method="ml")
# print(type(params.precipitable_water[:, 0, 0]))
params

<xarray.Dataset> Size: 2kB
Dimensions:                (id: 5, dparams: 3)
Coordinates: (12/15)
    drainage_area          (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date               (id) datetime64[ns] 40B 2006-10-13 ... 1996-08-13
  * id                     (id) object 40B '020302' '020404' ... '020802'
    latitude               (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude              (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name                   (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 40B 1989-08-12 ... 1970-01-01
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * dparams                (dparams) <U5 60B 'c' 'loc' 'scale'
Data variables:
    streamflow_max_spring  (id, dparams) float64 120B 1.26 359.5 ... 248.4 71.07
    streamflow_max_summer  (id, dparams) float64 120B -0.4456 46.99 ... 58.43
    streamflow_max_fall    (id, dparams) float64 120B -0.8061 33.16 ... 19.67
    streamflow_max_august  (id, dparams) float64 120B -0.9013 7.453 ... 10.99
    streamflow_max_annual  (id, dparams) float64 120B 1.504 401.7 ... 71.74
    streamflow_max_winter  (id, dparams) float64 120B 1.275 196.3 ... 16.65 10.7
    streamflow_max_custom  (id, dparams) float64 120B 0.3362 240.9 ... 75.95
    volume_sum_spring      (id, dparams) float64 120B 0.6526 322.2 ... 131.3
    volume_sum_annual      (id, dparams) float64 120B 0.2645 1.028e+03 ... 161.0
Attributes:
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:                
    long_name:             genextreme parameters
    description:           Parameters of the genextreme distribution
    method:                ML
    estimator:             Maximum likelihood
    scipy_dist:            genextreme
    units:                 
    history:               [2024-07-22 09:52:38] fit: Estimate distribution p...